<a href="https://colab.research.google.com/github/rudraser/Fluter-BasicApps/blob/main/Custom_ResNet_attempt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [2]:

from matplotlib import pyplot as plt
import numpy as np
import collections


import torch.nn as nn

import torch.optim as optim

from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),                        
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


Extracting /data-unversioned/p1ch6/cifar-10-python.tar.gz to /data-unversioned/p1ch6/


In [3]:
from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),                        
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([               #loading the val set
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
class resblock(nn.Module):                    #creating a resblock which would be used iterativealy.
    def __init__(self, n):
        super(resblock, self).__init__()
        self.conv = nn.Conv2d(n, n, kernel_size=3,
                              padding=1, bias=False)                 # defining layers for forward
        self.batch_norm = nn.BatchNorm2d(num_features=n)

        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)                                 #Actual resblock with the skip connection over 1 layer
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [6]:
class deep(nn.Module):                # the model which uses resblock iteratively and some other layers
    def __init__(self, n1=32, n_blocks=16):
        super().__init__()
        self.n1 = n1
        self.conv1 = nn.Conv2d(3, n1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n1, 2*n1, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(2*n1, 4*n1, kernel_size=3, padding=1)       # defing layers for forward
        self.resblocks = nn.Sequential(
            *(n_blocks * [resblock(n=n1)]))
        self.resblocks2 = nn.Sequential(
            *((n_blocks*2) * [resblock(n=n1*2)]))
        self.resblocks3 = nn.Sequential(
            *(n_blocks * [resblock(n=n1*4)]))
        self.fc1 = nn.Linear(8 * 8 * n1*4, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 10)
        
    def forward(self, x):                    # using the resblocks 3 times, each with twice the number of channels as before
        out = torch.relu(self.conv1(x))
        out = self.resblocks(out)                          
        out = F.max_pool2d(out, 2) 
        out = self.conv2(out)
        out = self.resblocks2(out)
        out = F.max_pool2d(out, 2) 
        out = self.conv3(out)
        out = self.resblocks3(out)
        out = out.view(-1, 8 * 8 * self.n1*4)
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [7]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):           # defining the training loop
    for epoch in range(1, n_epochs + 1):                                        # these inputs would be defined in next cell
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [8]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)                   # defining mini batch size and shuffling

model =  deep().to(device=device)# <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss()

In [9]:

training_loop(                   #calling the training loop finally
    n_epochs = 60,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


2021-07-31 08:09:15.113143 Epoch 1, Training loss 2.3090869444410513
2021-07-31 08:10:50.341553 Epoch 2, Training loss 2.3034862057327308
2021-07-31 08:12:26.276309 Epoch 3, Training loss 2.262720541118661
2021-07-31 08:14:03.904117 Epoch 4, Training loss 1.6089739499189664
2021-07-31 08:15:41.380519 Epoch 5, Training loss 1.279591747257106
2021-07-31 08:17:19.262187 Epoch 6, Training loss 1.083417211652107
2021-07-31 08:18:57.065606 Epoch 7, Training loss 0.9525606626135004
2021-07-31 08:20:34.655386 Epoch 8, Training loss 0.863658093621054
2021-07-31 08:22:12.602637 Epoch 9, Training loss 0.7995709673980312
2021-07-31 08:23:50.325300 Epoch 10, Training loss 0.7456416372600418
2021-07-31 08:25:28.016881 Epoch 11, Training loss 0.7095348755721851
2021-07-31 08:27:05.523823 Epoch 12, Training loss 0.6733422265638171
2021-07-31 08:28:43.299871 Epoch 13, Training loss 0.641565433891533
2021-07-31 08:30:21.326785 Epoch 14, Training loss 0.6166433519338403
2021-07-31 08:31:58.809628 Epoch 1

In [10]:

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():   # we do no need backpropagation for this hence the nograd
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # taking the class at maximum probability as prediction
                total += labels.shape[0]  
                correct += int((predicted == labels).sum())   # calculating the no. of correct predictions.

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.84
Accuracy val: 0.80


In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.005, weight_decay=0.01)

In [12]:
training_loop(  
    n_epochs = 10,                      
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2021-07-31 09:47:44.543312 Epoch 1, Training loss 0.22255751383407493
2021-07-31 09:49:23.068611 Epoch 2, Training loss 0.20387168931762886
2021-07-31 09:51:02.075728 Epoch 3, Training loss 0.20181098516525514
2021-07-31 09:52:40.531576 Epoch 4, Training loss 0.20029242577798226
2021-07-31 09:54:19.138255 Epoch 5, Training loss 0.1956471597818691
2021-07-31 09:55:57.830027 Epoch 6, Training loss 0.20162048854905626
2021-07-31 09:57:36.538567 Epoch 7, Training loss 0.2008804143513636
2021-07-31 09:59:15.848272 Epoch 8, Training loss 0.20314151855647716
2021-07-31 10:00:54.561053 Epoch 9, Training loss 0.19794660950998974
2021-07-31 10:02:33.216900 Epoch 10, Training loss 0.19925006257031885


In [13]:
validate(model, train_loader, val_loader)           #now checking acc

Accuracy train: 0.93
Accuracy val: 0.85


In [14]:
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.01)     #further lowering LR

In [15]:
training_loop(                    # another 10 iterations
    n_epochs = 15,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2021-07-31 10:04:59.235962 Epoch 1, Training loss 0.12006691121317618
2021-07-31 10:06:37.820553 Epoch 2, Training loss 0.09322239496849972
2021-07-31 10:08:15.739263 Epoch 3, Training loss 0.0818426752393908
2021-07-31 10:09:53.420914 Epoch 4, Training loss 0.07769588056041876
2021-07-31 10:11:31.027725 Epoch 5, Training loss 0.07096302416294699
2021-07-31 10:13:08.688152 Epoch 6, Training loss 0.06821863951942767
2021-07-31 10:14:45.853902 Epoch 7, Training loss 0.06539886328213089
2021-07-31 10:16:23.407596 Epoch 8, Training loss 0.06339500184752085
2021-07-31 10:18:00.951224 Epoch 9, Training loss 0.05940855986526822
2021-07-31 10:19:38.120934 Epoch 10, Training loss 0.058325695926728455
2021-07-31 10:21:15.571902 Epoch 11, Training loss 0.057290175416366294
2021-07-31 10:22:52.943547 Epoch 12, Training loss 0.05460345429842315
2021-07-31 10:24:30.102784 Epoch 13, Training loss 0.05364889797606908
2021-07-31 10:26:07.293617 Epoch 14, Training loss 0.053320423220855936
2021-07-31 10

In [16]:
validate(model, train_loader, val_loader)           #now checking acc

Accuracy train: 0.98
Accuracy val: 0.87


In [17]:
optimizer = optim.SGD(model.parameters(), lr=0.0005, weight_decay=0.001)     #further lowering LR

In [18]:
training_loop(                    # another 10 iterations
    n_epochs = 15,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2021-07-31 10:30:10.414984 Epoch 1, Training loss 0.04089769851380621
2021-07-31 10:31:47.311873 Epoch 2, Training loss 0.036981203542698335
2021-07-31 10:33:24.754653 Epoch 3, Training loss 0.03565228300715756
2021-07-31 10:35:02.013722 Epoch 4, Training loss 0.03268932851503038
2021-07-31 10:36:38.985878 Epoch 5, Training loss 0.03207675413028492
2021-07-31 10:38:16.209678 Epoch 6, Training loss 0.03214629590862652
2021-07-31 10:39:53.583887 Epoch 7, Training loss 0.031186605056049897
2021-07-31 10:41:30.780593 Epoch 8, Training loss 0.030017249009874472
2021-07-31 10:43:08.277706 Epoch 9, Training loss 0.02885091381342224
2021-07-31 10:44:45.665569 Epoch 10, Training loss 0.02699403008675355
2021-07-31 10:46:23.055295 Epoch 11, Training loss 0.025960669507864444
2021-07-31 10:48:00.250510 Epoch 12, Training loss 0.02544320775035178
2021-07-31 10:49:37.504836 Epoch 13, Training loss 0.0263497814374721
2021-07-31 10:51:14.854424 Epoch 14, Training loss 0.025131487349808083
2021-07-31 

In [19]:
validate(model, train_loader, val_loader)           #now checking acc

Accuracy train: 0.99
Accuracy val: 0.87
